In [2]:
import duckdb 
con = duckdb.connect(database='C:/Users/mimoune.djouallah/Desktop/TPC-H-SF10/db/tpch',read_only=True)

In [8]:
xx = con.execute('''

SELECT
    L_orderkey, 
    SUM(l_quantity) AS sum_qty
FROM
    lineitem
GROUP BY
    1
ORDER BY
    2 desc
    limit 501 ; 
''').df()
xx

,L_ORDERKEY,sum_qty
0,26465922,331.0
1,4806726,328.0
2,21213895,327.0
3,46685344,327.0
4,48881602,327.0
...,...,...
496,30338976,303.0
497,57968195,303.0
498,13841255,302.0
499,17612003,302.0


In [9]:
# Lineitem has  60 million records
xx = con.execute('''
SELECT
    l_returnflag,
    l_linestatus,
    SUM(l_quantity) AS sum_qty,
    SUM(l_extendedprice) AS sum_base_price,
    SUM(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    SUM(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    AVG(l_quantity) AS avg_qty,
    AVG(l_extendedprice) AS avg_price,
    AVG(l_discount) AS avg_disc
FROM
    lineitem
GROUP BY
    l_returnflag,
    l_linestatus
ORDER BY
    l_returnflag,
    l_linestatus;
    ''').df()
xx

,L_RETURNFLAG,L_LINESTATUS,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc
0,A,F,377518399.0,5.660657e+11,5.377591e+11,5.592767e+11,25.500975,38237.151009,0.050007
1,N,F,9851614.0,1.476744e+10,1.402881e+10,1.459049e+10,25.522448,38257.810660,0.049973
2,N,O,764635193.0,1.146549e+12,1.089216e+12,1.132797e+12,25.498214,38233.853934,0.050001
3,R,F,377732830.0,5.664311e+11,5.381109e+11,5.596348e+11,25.508385,38251.219274,0.049997


In [15]:
xx=con.execute('''

SELECT 
   customer_0."C_NAME" as "C_NAME",
   COALESCE(SUM(lineitem."L_QUANTITY"),0) as "sum_qty"
FROM '/Users/mimoune.djouallah/Desktop/TPC-H-SF10/Parquet/lineitem.parquet' as lineitem
LEFT JOIN '/Users/mimoune.djouallah/Desktop/TPC-H-SF10/Parquet/orders.parquet' AS orders_0
  ON orders_0."O_ORDERKEY"=lineitem."L_ORDERKEY"
LEFT JOIN '/Users/mimoune.djouallah/Desktop/TPC-H-SF10/Parquet/customer.parquet' AS customer_0
  ON customer_0."C_CUSTKEY"=orders_0."O_CUSTKEY"
GROUP BY 1
ORDER BY 2 desc NULLS LAST
LIMIT 501 ;
           ''').df()
xx

,C_NAME,sum_qty
0,Customer#000317698,4880.0
1,Customer#000694366,4756.0
2,Customer#000910705,4731.0
3,Customer#000750415,4716.0
4,Customer#000359587,4706.0
...,...,...
496,Customer#000684004,3873.0
497,Customer#000924955,3871.0
498,Customer#001432723,3870.0
499,Customer#001008862,3870.0


In [5]:
# the table is 60 million records, with 33 million unique text 
xx = con.execute('''
SELECT
    l_comment,
    SUM(l_quantity) AS sum_qty,
    SUM(l_extendedprice) AS sum_base_price,
    SUM(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    SUM(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    AVG(l_quantity) AS avg_qty,
    AVG(l_extendedprice) AS avg_price,
    AVG(l_discount) AS avg_disc
FROM
    lineitem
where l_comment like '%carefully%'
GROUP BY
    all
ORDER BY
    2  desc
limit 501 ;
    ''').df()
xx

,L_COMMENT,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc
0,carefully,233132.0,3.494738e+08,3.319834e+08,3.452452e+08,25.481692,38198.034950,0.050141
1,carefully,215171.0,3.218755e+08,3.057257e+08,3.181725e+08,25.361975,37939.115077,0.049894
2,carefully,213553.0,3.195423e+08,3.034478e+08,3.156221e+08,25.380675,37977.459308,0.050203
3,e carefully,104628.0,1.560138e+08,1.481716e+08,1.541446e+08,25.994534,38761.190435,0.050092
4,e carefully,88948.0,1.334310e+08,1.267860e+08,1.318291e+08,25.559770,38342.237221,0.049770
...,...,...,...,...,...,...,...,...
496,ost carefully,5252.0,7.634618e+06,7.277658e+06,7.606470e+06,24.427907,35509.849721,0.047860
497,ross the carefully,5249.0,7.828127e+06,7.441797e+06,7.751023e+06,27.056701,40351.171186,0.048454
498,the carefully s,5238.0,8.082824e+06,7.690452e+06,8.030897e+06,25.304348,39047.461014,0.049662
499,ages. carefully,5233.0,7.940068e+06,7.533151e+06,7.838192e+06,23.572072,35766.074144,0.050045


In [6]:
 
xx = con.execute('''

SELECT
    
    count (distinct L_orderkey)
    
FROM
    lineitem

 ;
    ''').df()
xx

,"count(DISTINCT ""L_orderkey"")"
0,15000000


In [32]:
# the table is 60 million records, with 33 million unique text 
xx = con.execute('''

SELECT
    l_shipdate ,
    count (distinct L_orderkey),
    
FROM
    lineitem
group by 1
order by 2 desc 
 ;
    ''').df()
xx

,L_SHIPDATE,"count(DISTINCT ""L_orderkey"")"
0,1993-07-10,25083
1,1997-04-18,25029
2,1993-05-17,25008
3,1994-01-17,24998
4,1993-04-01,24985
...,...,...
2521,1998-11-29,583
2522,1992-01-03,405
2523,1998-11-30,402
2524,1998-12-01,214


In [7]:
xx = con.execute('''

SELECT
    l_comment ,
    count (distinct L_orderkey),
    
FROM
    lineitem
group by 1
order by 2 desc 
limit 10
 ;
    ''').df()
xx

,L_COMMENT,"count(DISTINCT ""L_orderkey"")"
0,furiously,9519
1,carefully,9144
2,carefully,8483
3,furiously,8475
4,carefully,8411
5,furiously,8410
6,blithely,6633
7,ly regular,6030
8,ly regular,5751
9,y regular,5639


In [11]:
xx = con.execute('''
SELECT 
   customer_region_0."R_NAME" as "R_NAME",
   COALESCE(SUM(lineitem."L_QUANTITY"),0) as "sum_qty"
FROM '/Users/mimoune.djouallah/Desktop/TPC-H-SF10/Parquet/lineitem.parquet' as lineitem
LEFT JOIN '/Users/mimoune.djouallah/Desktop/TPC-H-SF10/Parquet/orders.parquet' AS orders_0
  ON orders_0."O_ORDERKEY"=lineitem."L_ORDERKEY"
LEFT JOIN '/Users/mimoune.djouallah/Desktop/TPC-H-SF10/Parquet/customer.parquet' AS customer_0
  ON customer_0."C_CUSTKEY"=orders_0."O_CUSTKEY"
LEFT JOIN '/Users/mimoune.djouallah/Desktop/TPC-H-SF10/Parquet/nation.parquet' AS customer_nation_0
  ON customer_nation_0."N_NATIONKEY"=customer_0."C_NATIONKEY"
LEFT JOIN '/Users/mimoune.djouallah/Desktop/TPC-H-SF10/Parquet/region.parquet' AS customer_region_0
  ON customer_region_0."R_REGIONKEY"=customer_nation_0."N_REGIONKEY"
GROUP BY 1
ORDER BY 2 desc NULLS LAST
    ''').df()
xx

,R_NAME,sum_qty
0,EUROPE,306686205.0
1,ASIA,306573761.0
2,AFRICA,305691519.0
3,MIDDLE EAST,305553071.0
4,AMERICA,305233480.0


In [9]:
import duckdb 
con = duckdb.connect()
xx = con.execute('''

SELECT
    l_comment ,
    count (distinct L_orderkey),
    
FROM 'C:/Users/mimoune.djouallah/Desktop/TPC-H-SF10/Parquet/lineitem/*.parquet'
group by 1
order by 2 desc 
limit 10
 ;
    ''').df()
xx

,L_COMMENT,"count(DISTINCT ""L_orderkey"")"
0,furiously,9519
1,carefully,9144
2,carefully,8483
3,furiously,8475
4,carefully,8411
5,furiously,8410
6,blithely,6633
7,ly regular,6030
8,ly regular,5751
9,y regular,5639
